In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os


In [2]:
input_file="diy_rejection_25-April'25.csv"
output_file="diy_mco_rejection_24th April'25.csv"

In [3]:
#input_file="diy_all_rejection_01st April.csv"
#output_file="diy_mco_all_rejection_31st Mar'25.csv"

In [4]:
rejection_data=pd.read_csv(input_file,low_memory=False,memory_map=True)
len(rejection_data)

20245

In [5]:
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1203526992,49b8f458-ba23-49e6-9de4-56fbc25effc9,PROPRIETORSHIP,Proprietor name is missing,udyam_udhyog_aadhaar,2025-04-04 17:54:03.000,diy_mco,business_name_update,NaN
1,1203526992,49b8f458-ba23-49e6-9de4-56fbc25effc9,PROPRIETORSHIP,Shop front not completely visible,shopFrontPhoto,2025-04-04 17:54:03.000,diy_mco,business_name_update,NaN
2,1295137127,9f25598a-0ce4-415e-a4cd-0ff4787c0c40,INDIVIDUAL,Beneficiary name/Account number/IFSC missing,bankPassbook,2025-04-18 17:54:53.000,diy_mco,NaN,NaN
3,600287255,3535b47f-5d92-4391-8cae-d14cbe01b53e,INDIVIDUAL,Complete Shop photo not uploaded,shopFrontPhoto,2025-04-20 20:42:53.000,diy_mco,registered_address_update,NaN
4,600287255,3535b47f-5d92-4391-8cae-d14cbe01b53e,INDIVIDUAL,Proprietor name is missing,udyam_udhyog_aadhaar,2025-04-20 20:42:53.000,diy_mco,registered_address_update,NaN


In [6]:
rejection_data['updated_at']=pd.to_datetime(rejection_data['updated_at'],errors='coerce')
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1203526992,49b8f458-ba23-49e6-9de4-56fbc25effc9,PROPRIETORSHIP,Proprietor name is missing,udyam_udhyog_aadhaar,2025-04-04 17:54:03,diy_mco,business_name_update,NaN
1,1203526992,49b8f458-ba23-49e6-9de4-56fbc25effc9,PROPRIETORSHIP,Shop front not completely visible,shopFrontPhoto,2025-04-04 17:54:03,diy_mco,business_name_update,NaN
2,1295137127,9f25598a-0ce4-415e-a4cd-0ff4787c0c40,INDIVIDUAL,Beneficiary name/Account number/IFSC missing,bankPassbook,2025-04-18 17:54:53,diy_mco,NaN,NaN
3,600287255,3535b47f-5d92-4391-8cae-d14cbe01b53e,INDIVIDUAL,Complete Shop photo not uploaded,shopFrontPhoto,2025-04-20 20:42:53,diy_mco,registered_address_update,NaN
4,600287255,3535b47f-5d92-4391-8cae-d14cbe01b53e,INDIVIDUAL,Proprietor name is missing,udyam_udhyog_aadhaar,2025-04-20 20:42:53,diy_mco,registered_address_update,NaN


In [7]:
current_date = pd.Timestamp(datetime.now().date())
current_date


Timestamp('2025-04-25 00:00:00')

In [8]:
rejection_data=rejection_data[rejection_data['updated_at'].dt.date != current_date]
len(rejection_data)

c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


19612

In [9]:
rejection_data=rejection_data[rejection_data['solution_type_level_2'].isin(['diy_upgrade','']) | rejection_data['solution_type_level_2'].isna()]
len(rejection_data)

10375

In [10]:
rejection_data.loc[~rejection_data['entity_type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'entity_type']='NON_PROPRIETORSHIP'
rejection_data['entity_type'].value_counts()

INDIVIDUAL            6268
PROPRIETORSHIP        4073
NON_PROPRIETORSHIP      34
Name: entity_type, dtype: int64

In [11]:
rejection_data.insert(rejection_data.columns.get_loc('entity_type')+1,'entuty_type_1',np.nan)
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [12]:
rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']
rejection_data['entuty_type_1'].value_counts()

INDIVIDUAL            6268
PROPRIETORSHIP        4073
NON_PROPRIETORSHIP      34
Name: entuty_type_1, dtype: int64

In [13]:
rejection_data['entuty_type_1']=rejection_data['entuty_type_1'].replace(['PROPRIETORSHIP','NON_PROPRIETORSHIP'],'NON_EDC')
rejection_data['entuty_type_1'].value_counts()

INDIVIDUAL    6268
NON_EDC       4107
Name: entuty_type_1, dtype: int64

In [14]:
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [15]:
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_MCO'
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (~rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_ReKYC'
rejection_data.loc[~(rejection_data['entity_type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [16]:
rejection_data.to_csv(output_file, index=False)